<a href="https://colab.research.google.com/github/Charles0823/ormi3/blob/main/python/%EC%98%A4%EB%A5%B4%EB%AF%B8_3%EA%B8%B0_Kcs_python_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 크롤링

In [ ]:
# select만 알고 있으면 됩니다. 다른건 나중에 전문적으로 할때

In [1]:
import requests
import bs4

In [2]:
print(requests.__version__)
print(bs4.__version__)

2.31.0
4.11.2


In [4]:
import requests

html = requests.get('https://www.paullab.co.kr/stock.html')
html

<Response [200]>

In [ ]:
print(type(html))
print(dir(html))
print(html.encoding)
# html.encoding = 'utf-8' # 한글 깨져 나오면 이 코드를 실행해주세요.
# html.headers
# html.json
# html.raw
html.text    # 우리가 사용하는 것은 text뿐입니다.

In [5]:
params = {'pa1': 'val1', 'pa2': 'value2'}
response = requests.get('http://www.paullab.co.kr', params=params)
# response.text
response.url

'https://paullab.co.kr/?pa1=val1&pa2=value2'

In [ ]:
import time

for i in range(10):
    requests.get('https://habitmaker.co.kr/')
    time.sleep(1)

In [7]:
import requests
from bs4 import BeautifulSoup

response = requests.get('http://www.paullab.co.kr/stock.html')
response.encoding = 'utf-8'
html = response.text

soup = BeautifulSoup(html, 'html.parser') # 원하는 문자열로 잘라줌
# soup

In [ ]:
print(soup.prettify()) # html 문서형식으로 출력

In [9]:
soup.title

<title>Document</title>

In [ ]:
soup.head

In [11]:
soup.select('#update')

[<span id="update">update : 20.12.30 / 해외 크롤링이 Block되어 있으므로 크롤링이 안되시는 분은 이 URL(http://paullab.synology.me/stock.html)을 사용하세요.</span>]

In [12]:
soup.select('#_market_sum')
soup.select('#_market_sum')[0]
soup.select('#_market_sum')[0].text

'349조 2,323'

In [20]:
# 2019.10.23 거래량 출력
soup.select('.table > tbody > tr')[1].select('td')[5].text # 높이 평가합니다.
# soup.select('.num span')[5].text
# 포털사는 클래스 이름을 주기적으로 변경하기 때문에 이 코드는 곧 작동되지 않습니다.

'6,580'

In [16]:
# 문제1 https://www.paullab.co.kr/stock.html 서비스를 크롤링하여 제주코딩베이스캠프 연구원의 2019년 10월 총 거래량을 구해주세요.

# 문제2 html에서 태그를 제거하고 텍스트만 뽑아낼 수 있도록 해주세요.
# '<p>This is a <em>simple</em> example.</p>'

# 문제3 아래 텍스트에서 이메일만 추출할 수 있는 정규표현식을 만들어주세요.
# '제 이메일 주소는 example1@gmail.com이고 회사 이메일 주소는 example2@gmail.com 입니다. example1로 연락을 주셨다면 제게 메시지 한 번 부탁드립니다.'

In [14]:
# 문제1 https://www.paullab.co.kr/stock.html 서비스를 크롤링하여 제주코딩베이스캠프 연구원의 2019년 10월 총 거래량을 구해주세요.

import requests
from bs4 import BeautifulSoup

response = requests.get('http://www.paullab.co.kr/stock.html')
response.encoding = 'utf-8'
html = response.text
soup = BeautifulSoup(html, 'html.parser')

# k = soup.select('.table > tbody > tr')[1].select('td')[6].text
all_trade = 0
day_trade = soup.select('.table > tbody > tr')[1:16]

for i in day_trade:
    all_trade += int(i.select('td')[6].text.replace(',',''))

print(all_trade)

10895826


In [15]:
# 강사님 풀이

request = requests.get('https://www.paullab.co.kr/stock.html')
html = request.text
soup = BeautifulSoup(html, 'html.parser')

i = 1
total = 0
while True:
    tr = soup.select('.main > table > tbody > tr')[i]
    month = int(tr.select('td')[0].text.split('.')[1])
    if month != 10:
        break
    sells = int(tr.select('td')[6].text.replace(',',''))
    total += sells
    i += 1
total

10895826

In [32]:
# 문제2 html에서 태그를 제거하고 텍스트만 뽑아낼 수 있도록 해주세요.

import re

s = '<p>This is a <em>simple</em> example.</p>'

text = re.sub(r'<[^>]*>', '', s)
print(text)

This is a simple example.


In [5]:
# 문제3 아래 텍스트에서 이메일만 추출할 수 있는 정규표현식을 만들어주세요.

import re

text = '제 이메일 주소는 example1@gmail.com이고 회사 이메일 주소는 example2@gmail.com 입니다. example1로 연락을 주셨다면 제게 메시지 한 번 부탁드립니다.'

k = re.sub(r'이고', '', text)
email = r'\S+@\S+'

emails = re.findall(email, k)
print(emails)

['example1@gmail.com', 'example2@gmail.com']


In [16]:
text = '제 이메일 주소는 example1@gmail.com이고 회사 이메일 주소는 example2@gmail.com 입니다. example1로 연락을 주셨다면 제게 메시지 한 번 부탁드립니다.'
p = re.compile('\w+@\w+.com')
p.findall(text)

['example1@gmail.com', 'example2@gmail.com']

In [2]:
import requests
from bs4 import BeautifulSoup
import re

# 문제1 https://www.paullab.co.kr/stock.html
# 서비스를 크롤링하여 제주코딩베이스캠프 연구원의 2019년 10월 총 거래량을 구해주세요.

request = requests.get('https://www.paullab.co.kr/stock.html')
html = request.text
soup = BeautifulSoup(html, 'html.parser')

In [3]:
class DailyTradingVolume:
    def __init__(self, 날짜, 종가, 전일비, 시가, 고가, 저가, 거래량):
        self.날짜 = 날짜
        self.종가 = 종가
        self.전일비 = 전일비
        self.시가 = 시가
        self.고가 = 고가
        self.저가 = 저가
        self.거래량 = 거래량

    def __str__(self):
        return f'{self.날짜}, {self.종가}'

    def __repr__(self):
        return f'{self.날짜}, {self.종가}'

class PublicCompany:
    def __init__(self, 시가총액, 상장주식수, 매출, 비용, 순익):
        self.시가총액 = 시가총액
        self.상장주식수 = 상장주식수
        self.매출 = 매출
        self.비용 = 비용
        self.순익 = 순익
        self.거래 = [] # 여기를 좀 더 고도화하여 연도별로 관리되게 하시는 편이 좋아보입니다.

제코베연구원 = PublicCompany(시가총액 = 100,
                       상장주식수 = 100,
                       매출 = 100,
                       비용 = 100,
                       순익 = 100)

제코베연구원.매출

100

In [4]:
import requests
from bs4 import BeautifulSoup
import re

# 문제1 https://www.paullab.co.kr/stock.html
# 서비스를 크롤링하여 제주코딩베이스캠프 연구원의 2019년 10월 총 거래량을 구해주세요.

request = requests.get('https://www.paullab.co.kr/stock.html')
html = request.text
soup = BeautifulSoup(html, 'html.parser')

In [5]:
soup.select('.main > table > tbody > tr')[1]       #
soup.select('.main > table > tbody > tr')[2]       #
soup.select('.main > table > tbody > tr')[-1]      #

<tr>
<td align="center"><span class="date">2019.09.24</span></td>
<td class="num"><span>2,065</span></td>
<td class="num">
<img alt="상승" height="6" src="ico_up.gif" style="margin-right:4px;" width="7"/><span>
                            30
                        </span>
</td>
<td class="num"><span>2,020</span></td>
<td class="num"><span>2,090</span></td>
<td class="num"><span>2,020</span></td>
<td class="num"><span>139,085</span></td>
</tr>

In [ ]:
soup.select('.main > table')[0]     # 모든 내역 다 긁어오기 때문에 엄청 깁니다.

In [7]:
'''
<tr>
<th scope="col">날짜</th>
<th scope="col">종가</th>
<th scope="col">전일비</th>
<th scope="col">시가</th>
<th scope="col">고가</th>
<th scope="col">저가</th>
<th scope="col">거래량</th>
</tr>
'''
# print(type(soup.select('.main > table')[0]))
# print(type(soup))
for i in soup.select('.main > table')[0].select('tr')[1:]:
    # print(i.select('td')[0].text)
    # 날짜 =
    # 종가 =
    # 전일비 =
    # DailyTradingVolume()

2019.10.23
2019.10.22
2019.10.21
2019.10.18
2019.10.17
2019.10.16
2019.10.15
2019.10.14
2019.10.11
2019.10.10
2019.10.08
2019.10.07
2019.10.04
2019.10.02
2019.10.01
2019.09.30
2019.09.27
2019.09.26
2019.09.25
2019.09.24


In [8]:
# 날짜, 종가, 전일비, 시가, 고가, 저가, 거래량
for i in soup.select('.main > table')[0].select('tr')[1:]:
    날짜 = i.select('td')[0].text
    종가 = i.select('td')[1].text
    전일비 = i.select('td')[2].text
    시가 = i.select('td')[3].text
    고가 = i.select('td')[4].text
    저가 = i.select('td')[5].text
    거래량 = i.select('td')[6].text
    제코베연구원.거래.append(DailyTradingVolume(날짜, 종가, 전일비, 시가, 고가, 저가, 거래량))

제코베연구원.거래

[2019.10.23, 6,650,
 2019.10.22, 6,630,
 2019.10.21, 6,820,
 2019.10.18, 6,430,
 2019.10.17, 5,950,
 2019.10.16, 5,930,
 2019.10.15, 5,640,
 2019.10.14, 5,380,
 2019.10.11, 5,040,
 2019.10.10, 5,100,
 2019.10.08, 5,050,
 2019.10.07, 4,940,
 2019.10.04, 5,010,
 2019.10.02, 4,920,
 2019.10.01, 5,010,
 2019.09.30, 5,000,
 2019.09.27, 5,010,
 2019.09.26, 5,060,
 2019.09.25, 5,060,
 2019.09.24, 5,330]

In [9]:
제코베연구원.거래[0].시가

'6,590'

In [10]:
{
    '201910':{
        '01': {# 201910으로 가공
                '시가': 3000, # 3000으로 가공
                '고가': 2000,
                '저가': 1000
                # ...
            },
        '02': {# 201910으로 가공
                '시가': 3000, # 3000으로 가공
                '고가': 2000,
                '저가': 1000
                # ...
            },
        },
    '201911':{
        '01': {# 201910으로 가공
                '시가': 3000, # 3000으로 가공
                '고가': 2000,
                '저가': 1000
                # ...
            },
    }
}

{'201910': {'01': {'시가': 3000, '고가': 2000, '저가': 1000},
  '02': {'시가': 3000, '고가': 2000, '저가': 1000}},
 '201911': {'01': {'시가': 3000, '고가': 2000, '저가': 1000}}}